In [ ]:
# 自动下载一些必要库
!pip install torch torchvision
!pip install numpy matplotlib seaborn pillow

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 dataset 导入
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

In [ ]:
# 使用torch函数加载数据集
trainset = datasets.CIFAR10(root='/mnt/ssd/dataset_CIFAR-10', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)

# 加载测试数据集
testset = datasets.CIFAR10(root='/mnt/ssd/dataset_CIFAR-10/cifar-10-batches-py', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=16, shuffle=False, num_workers=4)

# CIFAR-10类别
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# 检查是否有可用的GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义模型-使用预训练模型resnet18在CIFAR-10上图像分类
model = torchvision.models.resnet18(pretrained=True)
# 固定参数
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(512, 10)       # 修改全连接层的输出为10

model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#モデルの学習
train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []
epoch = 10

for i in range(epoch):
    print('-'*5, 'Epoch [{}/{}] start'.format(i, epoch-1), '-'*5)
    epoch_loss = 0
    epoch_accuracy = 0
    model.train()
    for image, target in trainloader:
        image, target = image.to(device), target.to(device)
        output = model(image).squeeze()
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (output.argmax(dim=1) == target).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)
        
    model.eval()

    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss = 0

        for image, target in testloader:

            # 入力、正解を GPU へ移動
            image, target = image.to(device), target.to(device)

            # モデルに入力を順伝播させ予測を出力
            output = model(image).squeeze()

            # 損失関数で予測と正解の誤差や精度を導出
            loss = criterion(output, target)

            acc = (output.argmax(dim=1) == target).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += loss / len(testloader)

    print(f'epoch:{i+1}/{epoch}, train_loss:{epoch_loss:.2f}, train_acc:{epoch_accuracy:.2f}, '+
          f'val_loss:{epoch_val_loss:.2f}, val_acc:{epoch_val_accuracy:.2f}')
    train_loss_list.append(epoch_loss)
    train_acc_list.append(epoch_accuracy)
    val_loss_list.append(epoch_val_loss)
    val_acc_list.append(epoch_val_accuracy)

In [ ]:
#学習曲線の描画
train_acc = []
train_loss = []
val_acc = []
val_loss = []

for i in range(epoch):
  train_acc2 = train_acc_list[i].cpu()
  train_acc3 = train_acc2.clone().numpy()
  train_acc.append(train_acc3)

  train_loss2 = train_loss_list[i].cpu()
  train_loss3 = train_loss2.detach().numpy()
  train_loss.append(train_loss3)

  val_acc2 = val_acc_list[i].cpu()
  val_acc3 = val_acc2.clone().numpy()
  val_acc.append(val_acc3)

  val_loss2 = val_loss_list[i].cpu()
  val_loss3 = val_loss2.clone().numpy()
  val_loss.append(val_loss3)


#取得したデータをグラフ化する
sns.set()
num_epochs = epoch
fig = plt.subplots(figsize=(12,4), dpi=80)

ax1 = plt.subplot(1,2,1)
ax1.plot(range(num_epochs), train_acc, c='b', label='train acc')
ax1.plot(range(num_epochs), val_acc, c='r', label='val acc')
ax1.set_xlabel('epoch', fontsize='12')
ax1.set_ylabel('accuracy', fontsize='12')
ax1.set_title('training and val acc', fontsize='14')
ax1.legend(fontsize='12')

ax2 = plt.subplot(1,2,2)
ax2.plot(range(num_epochs), train_loss, c='b', label='train loss')
ax2.plot(range(num_epochs), val_loss, c='r', label='val loss')
ax2.set_xlabel('epoch', fontsize='12')
ax2.set_ylabel('loss', fontsize='12')
ax2.set_title('training and val loss', fontsize='14')
ax2.legend(fontsize='12')